In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import ujson
import glob as g
import random as rnd
import os.path as op
import time
import math
import numpy as np
from skimage import io, transform
import skimage.color as skc

import multiprocessing
import hashlib as hl

In [2]:
line_paths = g.glob('/media/jakob/bigdata/train/S/*/*/*.bin.png')

In [3]:
len(line_paths)

58454

In [4]:
some_line_paths = line_paths[:400]

In [5]:
def read_img(path):
    img = io.imread(path)
    return img
def filter_img(img, std_height=48 ):
    height, length = img.shape
    return length >= std_height
def preprocess(img, std_height=48):
    rand = rnd.Random()
    height, length = img.shape
    norm_line = transform.resize(img,(std_height,length))
    offset = rand.randint(0, length - std_height)
    return np.reshape(norm_line[:, offset:offset + std_height],(1,-1))

In [10]:
plt.imshow(255 - preprocess(line_paths[400]),cmap='Greys')

AttributeError: 'str' object has no attribute 'shape'

In [8]:
X_data = np.concatenate(map(preprocess, filter(filter_img, map(read_img, some_line_paths))))
# X_data.shape
test_split = 0.7
# original_shape = 44,  66 , 3
tile_shape = 48,  48 
x_train = X_data[:int(len(X_data) * test_split)]
x_test  = X_data[int(len(X_data) * test_split):]

In [9]:
x_train.shape

(280, 2304)

In [48]:
from keras.layers import Input, Dense, Lambda,merge
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras.datasets import mnist

In [34]:
batch_size = 64
original_dim = x_train.shape[1]
latent_dim = 2
intermediate_dim = 128
epsilon_std = 0.01
nb_epoch = 40



In [70]:
def sampling(args):
#     print args.shape
#     z_mean, z_log_std = args[:,0:2], args[:,2:4]
    print args
    z_mean, z_log_std = args
    epsilon = K.random_normal(shape=(z_mean.shape[0], latent_dim),
                              mean=0., std=epsilon_std)
    return z_mean + K.exp(z_log_std) * epsilon

def vae_loss(x, x_decoded_mean):
    xent_loss = objectives.binary_crossentropy(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_std - K.square(z_mean) - K.exp(z_log_std), axis=-1)
    return xent_loss + kl_loss


x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_std = Dense(latent_dim)(h)


# merged = merge([z_mean, z_log_std])
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_std])
# z = z_layer()

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')

h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

vae = Model(x, x_decoded_mean)
vae.compile(optimizer='rmsprop', loss=vae_loss)

[Elemwise{add,no_inplace}.0, Elemwise{add,no_inplace}.0]


Exception: The `compute_mask` method of layer "z" should return one mask tensor per output tensor of the layer. Found: [None, None]

In [63]:
history = vae.fit(x_train, x_train,
        verbose=1,
        shuffle=False,
        nb_epoch=10,
        batch_size=batch_size,
        validation_data=(x_test, x_test))

Train on 280 samples, validate on 120 samples
Epoch 1/10


ValueError: Input dimension mis-match. (input[0].shape[1] = 2, input[2].shape[1] = 0)
Apply node that caused the error: Elemwise{Composite{(i0 + (i1 * i2 * i3))}}[(0, 0)](Subtensor{::, int64:int64:}.0, TensorConstant{(1, 1) of 0.01}, Elemwise{exp,no_inplace}.0, Reshape{2}.0)
Toposort index: 72
Inputs types: [TensorType(float32, matrix), TensorType(float32, (True, True)), TensorType(float32, matrix), TensorType(float32, matrix)]
Inputs shapes: [(64, 2), (1, 1), (64, 0), (64, 2)]
Inputs strides: [(8, 4), (4, 4), (4, 4), (8, 4)]
Inputs values: ['not shown', array([[ 0.01]], dtype=float32), array([], shape=(64, 0), dtype=float32), 'not shown']
Outputs clients: [[Dot22(Elemwise{Composite{(i0 + (i1 * i2 * i3))}}[(0, 0)].0, dense_74_W), InplaceDimShuffle{1,0}(Elemwise{Composite{(i0 + (i1 * i2 * i3))}}[(0, 0)].0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_